In [1]:
from pynndescent import NNDescent
import argparse
import time
from util import *
import sys
import faiss


In [2]:
space = 'l2'
feat_path = "/data/juelin/project/melee/data/datasets/bigann/base.1B.u8bin"
query_path = '/data/juelin/project/melee/data/datasets/bigann/query.10k.u8bin'
truth_path = '/data/juelin/project/melee/data/datasets/gt/GT_10M/bigann-10M'
index_path = '/data/juelin/project/melee/data/nnd/bigann/bigann_10M_M16.npy'
max_elements= 10000000

t = time.time()

feat = load_data(feat_path, max_elements, as_float=True)
query = load_data(query_path, sys.maxsize)
labels, distance = load_ground_truth(truth_path)

nnd_graph = np.load(index_path)

print(f"load feat in {time.time() - t} secs")

num_elements = feat.shape[0]
dim = feat.shape[1]
ids = np.arange(num_elements)
num_query = query.shape[0]


# index = NNDescent(data=feat, metric=metric, init_graph=nnd_graph, n_neighbors=nnd_graph.shape[1], low_memory=False, n_iters=0)
# index.prepare()
# print("START BENCHMARK")
# for k in [1, 10, 100]:
#     for epsilon in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35]:
#         t = time.time()
#         plabels, pdistance = index.query(query_data = query, k = k, epsilon=0.1)
#         total_matched = 0
#         for i in range(num_query):
#             gt_label = labels[i][:k]
#             p_label = plabels[i]
#             for pred in p_label:
#                 if pred in gt_label:
#                     total_matched += 1
#         recall = total_matched / (num_query * k)
#         print(f"{k=} {epsilon=} {recall=}", flush=True)
# print("END BENCHMARK")

load feat in 2.588117837905884 secs


In [9]:
nnd_graph = nnd_graph.reshape(-1, 16)
nnd_graph.shape

(10000000, 16)

In [12]:
metric = None
if space == "l2":
    metric = "euclidean"
elif space == "cosine" or "ip":
    metric = "cosine"

metric

'euclidean'

In [ ]:
index = NNDescent(data=feat, metric=metric, init_graph=nnd_graph, n_neighbors=nnd_graph.shape[1], low_memory=False, n_iters=0)
t = time.time()
print("START BENCHMARK")
for k in [1, 10, 100]:
    for epsilon in [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35]:
        t = time.time()
        plabels, pdistance = index.query(query_data = query, k = k, epsilon=epsilon)
        total_matched = 0
        for i in range(num_query):
            gt_label = labels[i][:k]
            p_label = plabels[i]
            for pred in p_label:
                if pred in gt_label:
                    total_matched += 1
        recall = total_matched / (num_query * k)
        print(f"{k=} {epsilon=} {recall=}", flush=True)
print("END BENCHMARK")

START BENCHMARK
k=1 epsilon=0 recall=0.0
k=1 epsilon=0.05 recall=0.1058
k=1 epsilon=0.1 recall=0.1866
k=1 epsilon=0.15 recall=0.2813
k=1 epsilon=0.2 recall=0.3992
k=1 epsilon=0.25 recall=0.5286
k=1 epsilon=0.3 recall=0.6602
k=1 epsilon=0.35 recall=0.7771
k=10 epsilon=0 recall=0.428
k=10 epsilon=0.05 recall=0.55938
k=10 epsilon=0.1 recall=0.66579
k=10 epsilon=0.15 recall=0.76413
k=10 epsilon=0.2 recall=0.84913
k=10 epsilon=0.25 recall=0.91372
k=10 epsilon=0.3 recall=0.95587
k=10 epsilon=0.35 recall=0.97945
k=100 epsilon=0 recall=0.747311
k=100 epsilon=0.05 recall=0.833647
k=100 epsilon=0.1 recall=0.90161
k=100 epsilon=0.15 recall=0.948068
k=100 epsilon=0.2 recall=0.975272


In [6]:
t = time.time()
metric = None
if space == "l2":
    metric = faiss.METRIC_L2
elif space == "cosine" or "ip":
    metric = faiss.METRIC_Lp

index = faiss.IndexNNDescentFlat(feat.shape[1], 32, metric)
index.add(feat)
print(f"train feat in {time.time() - t} secs")


train feat in 338.6449990272522 secs


In [12]:
D, I = index.search(x=query, k=1)

In [13]:
D

array([[253282.],
       [ 31796.],
       [ 46358.],
       ...,
       [ 91179.],
       [167297.],
       [ 51105.]], dtype=float32)

In [15]:
print("START BENCHMARK")
for k in [1, 10, 100]:
    for L in [1, 2, 4, 8, 16, 32, 64, 128, 256]:
        t = time.time()
        pdistance, plabels = index.query(query_data = query, k = k)
        total_matched = 0
        for i in range(num_query):
            gt_label = labels[i][:k]
            p_label = plabels[i]
            for pred in p_label:
                if pred in gt_label:
                    total_matched += 1
        recall = total_matched / (num_query * k)
        print(f"{k=} {epsilon=} {recall=}", flush=True)
print("END BENCHMARK")

(10000, 1)